# 🎯 NeMo Bangla ASR Fine-tuning Pipeline

**Fine-tuning pretrained Bangla Conformer ASR model on large audio dataset**

- **Model**: `hishab/titu_stt_bn_conformer_large`
- **Framework**: NVIDIA NeMo + PyTorch Lightning
- **Hardware**: Kaggle P100 GPU (16GB VRAM)
- **Audio**: Full 40+ minute files (NO chunking)
- **Batch Size**: 1 (one full audio file per batch)

---

## 📦 Setup and Installation

**First time setup**: Uncomment and run the installation cell below to install NeMo toolkit.

**Note**: If NeMo is already installed, skip the installation cell and run the imports cell directly.

In [ ]:
# Install NeMo and dependencies (UNCOMMENT AND RUN FIRST TIME ONLY)
# !pip install -q nemo_toolkit['all']
# !pip install -q soundfile librosa

# Verify installation
# import nemo
# import nemo.collections.asr as nemo_asr
# print(f"NeMo version: {nemo.__version__}")

In [ ]:
# Import required libraries
import os
import json
import numpy as np
import soundfile as sf
import librosa
import pandas as pd
import gc
import torch
from pathlib import Path
from tqdm.auto import tqdm
from typing import List, Tuple, Dict
import pytorch_lightning as pl
from omegaconf import OmegaConf, open_dict

# Import NeMo
import nemo
import nemo.collections.asr as nemo_asr

print("✅ All imports successful")
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🎯 NeMo version: {nemo.__version__}")
print(f"💾 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

✅ All imports successful
🔥 PyTorch version: 2.10.0+cu128
💾 CUDA available: True
   GPU: Tesla P100-PCIE-16GB
   VRAM: 17.06 GB


/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py:435: UserWarning: 
    Found GPU0 Tesla P100-PCIE-16GB which is of cuda capability 6.0.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (7.0) - (12.0)
    
  queued_call()
/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py:435: UserWarning: 
    Please install PyTorch with a following CUDA
    configurations:  12.6 following instructions at
    https://pytorch.org/get-started/locally/
    
  queued_call()
/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py:435: UserWarning: 
Tesla P100-PCIE-16GB with CUDA capability sm_60 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_70 sm_75 sm_80 sm_86 sm_90 sm_100 sm_120.
If you want to use the Tesla P100-PCIE-16GB GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  queued_call()


In [ ]:
# Configuration
# ⚠️ UPDATE THESE PATHS FOR YOUR KAGGLE ENVIRONMENT

# Input paths (Kaggle dataset)
AUDIO_DIR = "/kaggle/input/dl-sprint-4-0-bengali-long-form-speech-recognition/transcription/transcription/train/audio"  # Directory containing long audio files
TRANSCRIPT_DIR = "/kaggle/input/dl-sprint-4-0-bengali-long-form-speech-recognition/transcription/transcription/train/annotation"  # Directory containing .txt transcript files
TEST_AUDIO_DIR = "/kaggle/input/dl-sprint-4-0-bengali-long-form-speech-recognition/transcription/transcription/test/audio"  # Directory containing test audio files

# Output paths
OUTPUT_DIR = "/kaggle/working/processed_data"
MANIFEST_DIR = os.path.join(OUTPUT_DIR, "manifests")
CHECKPOINT_DIR = "/kaggle/working/checkpoints"
FINAL_MODEL_PATH = "/kaggle/working/nemo_bangla_asr_finetuned.nemo"
SUBMISSION_PATH = "/kaggle/working/"

# Audio parameters
SAMPLE_RATE = 16000  # Hz
MAX_AUDIO_DURATION = 3600  # Maximum audio duration in seconds (1 hour safety limit)

# Dataset parameters
USE_FIRST_50_PERCENT = True  # Use only first 50% of training data for faster testing

# Training parameters (OPTIMIZED for full audio files)
BATCH_SIZE = 1  # MUST be 1 for long audio files
MAX_EPOCHS = 10
LEARNING_RATE = 1e-5  # Lower LR for stability with long sequences
VAL_SPLIT = 0.1  # 10% validation
GRADIENT_ACCUMULATION = 8  # Simulate batch_size=8
INFERENCE_BATCH_SIZE = 1  # Process one full audio at a time

# Memory optimization settings
USE_GRADIENT_CHECKPOINTING = True
EMPTY_CACHE_EVERY_N_STEPS = 10

# Create directories
os.makedirs(MANIFEST_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

print("📁 Directory structure created")
print(f"  Manifests: {MANIFEST_DIR}")
print(f"  Checkpoints: {CHECKPOINT_DIR}")
print(f"  Submission: {SUBMISSION_PATH}")
print("\n⚡ Processing FULL audio files (NO chunking)")
print(f"💾 Batch size: {BATCH_SIZE} (one full audio per batch)")
print(f"💾 Memory optimizations: Gradient accumulation={GRADIENT_ACCUMULATION}, Checkpointing={USE_GRADIENT_CHECKPOINTING}")
if USE_FIRST_50_PERCENT:
    print(f"📊 Dataset: Using first 50% of training data only")

📁 Directory structure created
  Manifests: /kaggle/working/processed_data/manifests
  Checkpoints: /kaggle/working/checkpoints
  Submission: /kaggle/working/

⚡ Using ON-THE-FLY chunking (no intermediate files saved)
💾 Memory optimizations: Gradient accumulation=4, Checkpointing=True
📊 Dataset: Using first 50% of training data only


### 📁 Expected Dataset Structure

```
/kaggle/input/your-dataset/
├── audio/
│   ├── audio1.wav  (~40 minutes long)
│   ├── audio2.wav  (~40 minutes long)
│   └── ...
├── transcripts/
│   ├── audio1.txt  ← FULL transcript for entire audio1.wav
│   ├── audio2.txt  ← FULL transcript for entire audio2.wav
│   └── ...
└── test/
    ├── test1.wav
    ├── test2.wav
    └── ...
```

**Important Notes**:
- Each audio file (~40 minutes) must have ONE corresponding .txt file with the SAME filename
- Each .txt file contains the COMPLETE transcript for the entire audio file
- NO chunking or splitting - full audio trained with full transcript

### ⚙️ Dataset Size Configuration

**Current setting**: `USE_FIRST_50_PERCENT = True`

- ✅ **True**: Uses only the **first 50%** of audio files for training (faster, less memory)
- ❌ **False**: Uses **all 100%** of audio files for training (full dataset, better accuracy)

This is useful for:
- Quick testing and iteration with smaller dataset
- Reducing training time and memory usage
- Prototyping before full training run

In [4]:
# Validate paths (optional but recommended)
print("🔍 Validating dataset paths...\n")

paths_to_check = {
    "Audio Directory": AUDIO_DIR,
    "Transcript Directory": TRANSCRIPT_DIR,
    "Test Audio Directory": TEST_AUDIO_DIR
}

all_valid = True
for name, path in paths_to_check.items():
    exists = os.path.exists(path)
    status = "✅" if exists else "❌"
    print(f"{status} {name}: {path}")
    if not exists:
        all_valid = False
        print(f"   ⚠️  WARNING: Path does not exist!")

if not all_valid:
    print("\n⚠️  Some paths are invalid. Please update the configuration above.")
    print("   The notebook will still run, but you may encounter errors later.")
else:
    print("\n✅ All paths validated successfully!")

🔍 Validating dataset paths...

✅ Audio Directory: /kaggle/input/dl-sprint-4-0-bengali-long-form-speech-recognition/transcription/transcription/train/audio
✅ Transcript Directory: /kaggle/input/dl-sprint-4-0-bengali-long-form-speech-recognition/transcription/transcription/train/annotation
✅ Test Audio Directory: /kaggle/input/dl-sprint-4-0-bengali-long-form-speech-recognition/transcription/transcription/test/audio

✅ All paths validated successfully!


---

## 🎵 Audio Preprocessing

Process full audio files with their complete transcripts (NO chunking).

In [ ]:
def get_audio_duration(audio_path: str) -> float:
    """
    Get audio file duration in seconds.
    
    Args:
        audio_path: Path to audio file
        
    Returns:
        Duration in seconds
    """
    info = sf.info(audio_path)
    return info.duration


def preprocess_audio(audio_path: str, target_sr: int = 16000) -> Tuple[np.ndarray, int]:
    """
    Load and preprocess audio file.
    
    Args:
        audio_path: Path to audio file
        target_sr: Target sample rate
        
    Returns:
        Tuple of (audio_data, sample_rate)
    """
    # Load audio
    audio, sr = sf.read(audio_path)
    
    # Resample if needed
    if sr != target_sr:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=target_sr)
        sr = target_sr
    
    # Convert stereo to mono if needed
    if len(audio.shape) > 1:
        audio = np.mean(audio, axis=1)
    
    # Normalize audio to [-1, 1]
    if audio.dtype != np.float32:
        audio = audio.astype(np.float32)
    
    max_val = np.abs(audio).max()
    if max_val > 0:
        audio = audio / max_val
    
    return audio, sr


print("✅ Audio preprocessing functions defined")

✅ Audio chunking functions defined (offset-based, no file saving)


In [ ]:
def process_dataset(
    audio_dir: str,
    transcript_data: Dict[str, str],
    max_duration: float = 3600.0,
    use_first_n_percent: float = 1.0
) -> List[Dict]:
    """
    Process all audio files: pair full audio files with their full transcripts.
    NO CHUNKING - uses complete audio files.
    
    Args:
        audio_dir: Directory containing audio files
        transcript_data: Dictionary mapping audio files to FULL transcripts
                        Format: {"filename.wav": "full transcript text", ...}
        max_duration: Maximum audio duration to include (safety limit)
        use_first_n_percent: Fraction of data to use (0.5 = first 50%)
        
    Returns:
        List of dictionaries with audio_filepath and text
    """
    all_data = []
    skipped_too_long = 0
    skipped_no_transcript = 0
    
    # Get all audio files
    audio_files = sorted(Path(audio_dir).glob("*.wav"))
    
    # Take only first N% if specified
    if use_first_n_percent < 1.0:
        num_files_to_use = int(len(audio_files) * use_first_n_percent)
        audio_files = audio_files[:num_files_to_use]
        print(f"📊 Using first {use_first_n_percent*100:.0f}% of data: {num_files_to_use}/{len(sorted(Path(audio_dir).glob('*.wav')))} files")
    
    print(f"Found {len(audio_files)} audio files to process\n")
    
    for audio_path in tqdm(audio_files, desc="Processing audio files"):
        filename = audio_path.name
        
        # Check if transcript exists
        if filename not in transcript_data:
            print(f"⚠️  Skipping {filename}: no transcript found")
            skipped_no_transcript += 1
            continue
        
        # Get full transcript
        full_transcript = transcript_data[filename]
        
        # Validate transcript
        if not full_transcript or not full_transcript.strip():
            print(f"⚠️  Skipping {filename}: empty transcript")
            skipped_no_transcript += 1
            continue
        
        # Check audio duration
        try:
            duration = get_audio_duration(str(audio_path))
            
            if duration > max_duration:
                print(f"⚠️  Skipping {filename}: too long ({duration/60:.1f} min > {max_duration/60:.1f} min limit)")
                skipped_too_long += 1
                continue
        except Exception as e:
            print(f"⚠️  Skipping {filename}: error reading audio - {str(e)}")
            continue
        
        # Add to dataset (full audio + full transcript)
        all_data.append({
            "audio_filepath": str(audio_path),
            "duration": duration,
            "text": full_transcript.strip()
        })
    
    print(f"\n✅ Processed {len(all_data)} valid audio-transcript pairs")
    print(f"   Skipped: {skipped_no_transcript} (no/empty transcript), {skipped_too_long} (too long)")
    print(f"   Total audio duration: {sum(item['duration'] for item in all_data) / 3600:.2f} hours")
    return all_data


print("✅ Dataset processing function defined")

✅ Dataset processing function defined


### Load Transcripts from .txt Files

**Format**: Each audio file has a corresponding .txt file with the same name.
- Audio: `audio1.wav` (40 minutes) → Transcript: `audio1.txt` (full transcript)
- Each .txt file contains the COMPLETE transcript for the entire audio file
- NO splitting or chunking - full transcript used as-is for training

In [7]:
def load_transcript_from_file(transcript_path: str) -> str:
    """
    Load transcript from a .txt file.
    
    Args:
        transcript_path: Path to transcript .txt file
        
    Returns:
        Transcript text as string
    """
    try:
        with open(transcript_path, 'r', encoding='utf-8') as f:
            transcript = f.read().strip()
        return transcript
    except Exception as e:
        print(f"⚠️  Error reading {transcript_path}: {str(e)}")
        return ""


def load_transcripts_from_directory(
    audio_dir: str,
    transcript_dir: str
) -> Dict[str, str]:
    """
    Load all transcripts from directory.
    Matches audio files (*.wav) with transcript files (*.txt).
    
    Args:
        audio_dir: Directory containing audio files
        transcript_dir: Directory containing transcript .txt files
        
    Returns:
        Dictionary mapping audio filename to full transcript text
        Format: {"audio1.wav": "full transcript text", ...}
    """
    transcript_dict = {}
    
    # Get all audio files
    audio_files = sorted(Path(audio_dir).glob("*.wav"))
    
    print(f"📂 Loading transcripts from: {transcript_dir}")
    print(f"   Found {len(audio_files)} audio files in {audio_dir}")
    
    found_count = 0
    missing_count = 0
    
    for audio_path in audio_files:
        audio_filename = audio_path.name
        # Get corresponding transcript file (same name, .txt extension)
        transcript_filename = audio_path.stem + ".txt"
        transcript_path = Path(transcript_dir) / transcript_filename
        
        if transcript_path.exists():
            transcript = load_transcript_from_file(str(transcript_path))
            if transcript:
                transcript_dict[audio_filename] = transcript
                found_count += 1
            else:
                print(f"⚠️  Empty transcript: {transcript_filename}")
                missing_count += 1
        else:
            print(f"⚠️  Transcript not found: {transcript_filename}")
            missing_count += 1
    
    print(f"\n✅ Loaded {found_count} transcripts")
    if missing_count > 0:
        print(f"⚠️  Missing/empty: {missing_count} transcripts")
    
    return transcript_dict


print("✅ Transcript loading functions defined")

✅ Transcript loading functions defined


In [8]:
# Load transcripts from directory
transcript_data = load_transcripts_from_directory(
    audio_dir=AUDIO_DIR,
    transcript_dir=TRANSCRIPT_DIR
)

print(f"\n📊 Loaded transcripts for {len(transcript_data)} audio files")

📂 Loading transcripts from: /kaggle/input/dl-sprint-4-0-bengali-long-form-speech-recognition/transcription/transcription/train/annotation
   Found 113 audio files in /kaggle/input/dl-sprint-4-0-bengali-long-form-speech-recognition/transcription/transcription/train/audio

✅ Loaded 113 transcripts

📊 Loaded transcripts for 113 audio files


In [9]:
# Display sample transcripts
print("\n📄 Sample transcripts:")
print("-" * 80)

sample_count = min(3, len(transcript_data))
for i, (filename, transcript) in enumerate(list(transcript_data.items())[:sample_count]):
    print(f"\n{i+1}. File: {filename}")
    print(f"   Transcript length: {len(transcript)} characters")
    print(f"   Preview: {transcript[:150]}...")
    print("-" * 80)

if len(transcript_data) > sample_count:
    print(f"\n... and {len(transcript_data) - sample_count} more files")


📄 Sample transcripts:
--------------------------------------------------------------------------------

1. File: train_001.wav
   Transcript length: 29326 characters
   Preview: গল্পতর চ্যানেলে আপনাদের সাথে আছি আমি রাজ পড়ছিলাম হুমায়ুন আহমেদের উপন্যাস মধ্যান্য পড়ছি এর চতুর্থ পর্ব রঙিলা নটিবাড়ি সোহাগঞ্জ বাজারের শেষ মাথায় মা...
--------------------------------------------------------------------------------

2. File: train_002.wav
   Transcript length: 28494 characters
   Preview: গল্পতরু চ্যানেলে আপনাদের সাথে আছি আমি রাজ পড়ছিলাম হুমায়ুন আহমেদের উপন্যাস মধ্যান্য আজ পড়ছি এর ষষ্ঠ পর্ব হরিচরণ টিনের চালাঘরে খাটের উপর বসেছিলেন বাই...
--------------------------------------------------------------------------------

3. File: train_003.wav
   Transcript length: 36648 characters
   Preview: নিস্তব্ধতা এবং প্রাকৃতিক সৌন্দর্যের মাঝে ঘুমিয়ে আছে একটা গ্রাম পল্লী বাংলার আর পাঁচটা গ্রামের মতোই শান্তশিষ্ট ছোট্ট গ্রাম নাম তিনজলা চারপাশে ছোট ছোট ...
-----------------------------------------------

In [ ]:
# Process the dataset (NO FILES SAVED!)
# This creates offset-based metadata only

all_data = process_dataset(
    audio_dir=AUDIO_DIR,
    transcript_data=transcript_data,
    use_first_n_percent=0.5 if USE_FIRST_50_PERCENT else 1.0
)

print(f"\n📊 Dataset Statistics:")
print(f"  Total samples: {len(all_data)}")
print(f"  Total duration: {sum(item['duration'] for item in all_data) / 3600:.2f} hours")
print(f"\n📝 Sample data point (offset-based):")
if all_data:
    print(json.dumps(all_data[0], indent=2, ensure_ascii=False))
    print(f"\n  Sample transcript length: {len(all_data[0]['text'])} characters")
    print(f"  Sample transcript preview: {all_data[0]['text'][:150]}...")

📊 Using first 50% of data: 56/113 files
Found 56 audio files to process


Processing audio files:   0%|          | 0/56 [00:00<?, ?it/s]

⚠️  No transcript chunk for audio chunk 1 of train_001.wav
⚠️  Empty transcript for chunk 1 of train_001.wav
⚠️  No transcript chunk for audio chunk 2 of train_001.wav
⚠️  Empty transcript for chunk 2 of train_001.wav
⚠️  No transcript chunk for audio chunk 3 of train_001.wav
⚠️  Empty transcript for chunk 3 of train_001.wav
⚠️  No transcript chunk for audio chunk 4 of train_001.wav
⚠️  Empty transcript for chunk 4 of train_001.wav
⚠️  No transcript chunk for audio chunk 5 of train_001.wav
⚠️  Empty transcript for chunk 5 of train_001.wav
⚠️  No transcript chunk for audio chunk 6 of train_001.wav
⚠️  Empty transcript for chunk 6 of train_001.wav
⚠️  No transcript chunk for audio chunk 7 of train_001.wav
⚠️  Empty transcript for chunk 7 of train_001.wav
⚠️  No transcript chunk for audio chunk 8 of train_001.wav
⚠️  Empty transcript for chunk 8 of train_001.wav
⚠️  No transcript chunk for audio chunk 9 of train_001.wav
⚠️  Empty transcript for chunk 9 of train_001.wav
⚠️  No transcript c

---

## 📄 Manifest Generation

Create NeMo-compatible JSON manifest files for full audio files.

In [ ]:
def create_manifest(
    data: List[Dict],
    manifest_path: str,
    validate: bool = True
) -> None:
    """
    Create NeMo manifest file with offset support.
    
    Args:
        data: List of dicts with audio_filepath, offset, duration, text
        manifest_path: Output manifest file path
        validate: Whether to validate data before writing
    """
    valid_count = 0
    invalid_count = 0
    
    with open(manifest_path, 'w', encoding='utf-8') as f:
        for item in data:
            # Validation
            if validate:
                # Check audio file exists
                if not os.path.exists(item['audio_filepath']):
                    print(f"⚠️  Audio file not found: {item['audio_filepath']}")
                    invalid_count += 1
                    continue
                
                # Check text is not empty
                if not item['text'] or not item['text'].strip():
                    print(f"⚠️  Empty text for: {item['audio_filepath']}")
                    invalid_count += 1
                    continue
                
                # Check duration is positive
                if item['duration'] <= 0:
                    print(f"⚠️  Invalid duration for: {item['audio_filepath']}")
                    invalid_count += 1
                    continue
            
            # Write to manifest (one JSON per line)
            json_line = json.dumps(item, ensure_ascii=False)
            f.write(json_line + '\n')
            valid_count += 1
    
    print(f"\n✅ Manifest created: {manifest_path}")
    print(f"  Valid entries: {valid_count}")
    print(f"  Format: Full audio files (no chunking)")
    if invalid_count > 0:
        print(f"  ⚠️  Skipped invalid entries: {invalid_count}")


def train_val_split(
    data: List[Dict],
    val_ratio: float = 0.1,
    shuffle: bool = True,
    seed: int = 42
) -> Tuple[List[Dict], List[Dict]]:
    """
    Split data into train and validation sets.
    
    Args:
        data: List of data items
        val_ratio: Fraction of data for validation
        shuffle: Whether to shuffle before splitting
        seed: Random seed for reproducibility
        
    Returns:
        (train_data, val_data) tuple
    """
    if shuffle:
        np.random.seed(seed)
        indices = np.random.permutation(len(data))
        data = [data[i] for i in indices]
    
    split_idx = int(len(data) * (1 - val_ratio))
    train_data = data[:split_idx]
    val_data = data[split_idx:]
    
    return train_data, val_data


print("✅ Manifest functions defined")

✅ Manifest functions defined


In [12]:
# Memory management utilities
def clear_memory():
    """Clear GPU and CPU memory cache"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

def print_memory_stats():
    """Print current GPU memory usage"""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated(0) / 1e9
        reserved = torch.cuda.memory_reserved(0) / 1e9
        print(f"💾 GPU Memory: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved")

print("✅ Memory management utilities defined")

✅ Memory management utilities defined


In [13]:
# Split data into train/val
train_data, val_data = train_val_split(
    all_data,
    val_ratio=VAL_SPLIT,
    shuffle=True,
    seed=42
)

print(f"📊 Data Split:")
print(f"  Training samples: {len(train_data)}")
print(f"  Validation samples: {len(val_data)}")
print(f"  Train duration: {sum(item['duration'] for item in train_data) / 3600:.2f} hours")
print(f"  Val duration: {sum(item['duration'] for item in val_data) / 3600:.2f} hours")

📊 Data Split:
  Training samples: 2027
  Validation samples: 226
  Train duration: 16.89 hours
  Val duration: 1.88 hours


In [14]:
# Create manifest files
train_manifest_path = os.path.join(MANIFEST_DIR, "train_manifest.json")
val_manifest_path = os.path.join(MANIFEST_DIR, "val_manifest.json")

create_manifest(train_data, train_manifest_path, validate=True)
create_manifest(val_data, val_manifest_path, validate=True)

print(f"\n✅ Manifests ready:")
print(f"  Train: {train_manifest_path}")
print(f"  Val: {val_manifest_path}")


✅ Manifest created: /kaggle/working/processed_data/manifests/train_manifest.json
  Valid entries: 2027
  Format: Offset-based (no chunk files needed)

✅ Manifest created: /kaggle/working/processed_data/manifests/val_manifest.json
  Valid entries: 226
  Format: Offset-based (no chunk files needed)

✅ Manifests ready:
  Train: /kaggle/working/processed_data/manifests/train_manifest.json
  Val: /kaggle/working/processed_data/manifests/val_manifest.json


In [ ]:
# Optional: Verify manifest samples
print("🔍 Verifying manifest samples...\n")

def verify_manifest_samples(manifest_path, num_samples=3):
    """Verify that manifest entries can be loaded correctly"""
    print(f"Checking: {os.path.basename(manifest_path)}")
    
    with open(manifest_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    valid = 0
    invalid = 0
    
    for i, line in enumerate(lines[:num_samples]):
        try:
            data = json.loads(line)
            
            # Check required fields (no offset needed for full files)
            required = ['audio_filepath', 'duration', 'text']
            missing = [field for field in required if field not in data]
            
            if missing:
                print(f"  ❌ Entry {i+1}: Missing fields {missing}")
                invalid += 1
                continue
            
            # Check audio file exists
            if not os.path.exists(data['audio_filepath']):
                print(f"  ❌ Entry {i+1}: Audio file not found: {data['audio_filepath']}")
                invalid += 1
                continue
            
            # Check text is not empty
            if not data['text'].strip():
                print(f"  ⚠️  Entry {i+1}: Empty text")
                invalid += 1
                continue
            
            print(f"  ✅ Entry {i+1}: OK")
            print(f"     Audio: {os.path.basename(data['audio_filepath'])}")
            print(f"     Duration: {data['duration']/60:.1f} minutes")
            print(f"     Text: {data['text'][:100]}...")
            valid += 1
            
        except Exception as e:
            print(f"  ❌ Entry {i+1}: Error - {str(e)}")
            invalid += 1
    
    print(f"\n  Summary: {valid} valid, {invalid} invalid\n")

# Verify both manifests
verify_manifest_samples(train_manifest_path, num_samples=3)
verify_manifest_samples(val_manifest_path, num_samples=2)

print("✅ Manifest verification complete")

🔍 Verifying manifest samples...

Checking: train_manifest.json
  ✅ Entry 1: OK
     Audio: train_052.wav
     Offset: 2250.0s, Duration: 30.0s
     Text: তার নাম মদনাসুর ব্রহ্মার বরে সে অর্জন করেছিল বাক স...
  ✅ Entry 2: OK
     Audio: train_035.wav
     Offset: 2520.0s, Duration: 30.0s
     Text: আমাদের পায়ের কাছে একটা সাদা বিড়াল বসে আছে আমাদের...
  ✅ Entry 3: OK
     Audio: train_060.wav
     Offset: 30.0s, Duration: 30.0s
     Text: সাহিত্যের সব ধরনের ধারাতেই তিনি গল্প ও উপন্যাস লিখ...

  Summary: 3 valid, 0 invalid

Checking: val_manifest.json
  ✅ Entry 1: OK
     Audio: train_052.wav
     Offset: 990.0s, Duration: 30.0s
     Text: >> কি অপূর্ব মায়ের এই তারার রূপ রূপে ভয়ঙ্করী হয়...
  ✅ Entry 2: OK
     Audio: train_035.wav
     Offset: 210.0s, Duration: 30.0s
     Text: তারপর একটা চিঠি দিলেন আমাদের দিকে >> পড়ে দেখো লন্ঠ...

  Summary: 2 valid, 0 invalid

✅ Manifest verification complete


---

## 🧠 Model Fine-tuning

Load pretrained model and configure training for Kaggle P100.

In [16]:
# Load pretrained model
print("📥 Loading pretrained model: hishab/titu_stt_bn_conformer_large")
print("   This may take a few minutes...\n")

asr_model = nemo_asr.models.ASRModel.from_pretrained(
    "hishab/titu_stt_bn_conformer_large"
)

print("✅ Model loaded successfully")
print(f"   Model type: {type(asr_model).__name__}")
print(f"   Sample rate: {asr_model._cfg.sample_rate}")

📥 Loading pretrained model: hishab/titu_stt_bn_conformer_large
   This may take a few minutes...



NameError: name 'nemo_asr' is not defined

### 💡 Memory Troubleshooting Tips

If you encounter Out-of-Memory (OOM) errors:

**During Training:**
- Increase `GRADIENT_ACCUMULATION` to 8 or 16
- Keep `BATCH_SIZE = 1`
- Set `USE_GRADIENT_CHECKPOINTING = True`
- Reduce `MAX_EPOCHS` for faster testing

**During Inference:**
- Reduce `INFERENCE_BATCH_SIZE` to 2 or 1
- Reduce `CHUNK_DURATION` to 15 or 20 seconds
- Process test files one at a time (set batch_size=1 in transcribe function)

**General:**
- Restart kernel to clear all memory
- Close other notebooks/processes
- Monitor memory with `print_memory_stats()`

In [ ]:
# Configure training data
print("⚙️  Configuring training data...")

# Update training data config
with open_dict(asr_model.cfg):
    asr_model.cfg.train_ds.manifest_filepath = train_manifest_path
    asr_model.cfg.train_ds.batch_size = BATCH_SIZE
    asr_model.cfg.train_ds.shuffle = True
    asr_model.cfg.train_ds.num_workers = 2
    asr_model.cfg.train_ds.pin_memory = False  # Disabled for memory efficiency
    asr_model.cfg.train_ds.sample_rate = SAMPLE_RATE
    
    # IMPORTANT: Full audio file processing (no offset/chunking)
    asr_model.cfg.train_ds.use_start_end_token = False
    
    # Update validation data config
    asr_model.cfg.validation_ds.manifest_filepath = val_manifest_path
    asr_model.cfg.validation_ds.batch_size = BATCH_SIZE
    asr_model.cfg.validation_ds.shuffle = False
    asr_model.cfg.validation_ds.num_workers = 2
    asr_model.cfg.validation_ds.sample_rate = SAMPLE_RATE
    
    # Optimizer config
    asr_model.cfg.optim.name = 'adam'
    asr_model.cfg.optim.lr = LEARNING_RATE
    asr_model.cfg.optim.betas = [0.9, 0.999]
    asr_model.cfg.optim.weight_decay = 1e-6
    
    # Learning rate schedule (warmup + hold + decay)
    asr_model.cfg.optim.sched.name = 'CosineAnnealing'
    asr_model.cfg.optim.sched.warmup_steps = 500
    asr_model.cfg.optim.sched.min_lr = 1e-7
    
    # Memory optimization: Enable gradient checkpointing if available
    if USE_GRADIENT_CHECKPOINTING and hasattr(asr_model.cfg, 'encoder'):
        if hasattr(asr_model.cfg.encoder, 'gradient_checkpointing'):
            asr_model.cfg.encoder.gradient_checkpointing = True
            print("✅ Gradient checkpointing enabled")

# Setup training and validation data loaders
asr_model.setup_training_data(asr_model.cfg.train_ds)
asr_model.setup_validation_data(asr_model.cfg.validation_ds)

# Clear memory before training
clear_memory()
print_memory_stats()

print("✅ Data configuration complete")
print("⚡ Data loaders will read audio chunks on-the-fly using offsets")

In [ ]:
# Configure PyTorch Lightning Trainer
print("⚙️  Configuring trainer...")

from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, Callback
from pytorch_lightning.loggers import TensorBoardLogger

# Custom callback for memory management
class MemoryCleanupCallback(Callback):
    """Clear GPU cache periodically during training"""
    def __init__(self, cleanup_every_n_steps=10):
        self.cleanup_every_n_steps = cleanup_every_n_steps
    
    def on_train_batch_end(self, trainer, pl_module, outputs, batch, batch_idx):
        if batch_idx % self.cleanup_every_n_steps == 0:
            clear_memory()

# Checkpoint callback
checkpoint_callback = ModelCheckpoint(
    dirpath=CHECKPOINT_DIR,
    filename='nemo-asr-{epoch:02d}-{val_wer:.4f}',
    monitor='val_wer',
    mode='min',
    save_top_k=2,  # Reduced to save disk space
    save_last=True,
    verbose=True
)

# Early stopping (optional)
early_stop_callback = EarlyStopping(
    monitor='val_wer',
    patience=3,
    mode='min',
    verbose=True
)

# Memory cleanup callback
memory_callback = MemoryCleanupCallback(cleanup_every_n_steps=EMPTY_CACHE_EVERY_N_STEPS)

# Logger
logger = TensorBoardLogger(
    save_dir='/kaggle/working',
    name='nemo_asr_logs'
)

# Trainer
trainer = pl.Trainer(
    devices=1,
    accelerator='gpu',
    max_epochs=MAX_EPOCHS,
    precision='16-mixed',  # Mixed precision for P100
    callbacks=[checkpoint_callback, early_stop_callback, memory_callback],
    logger=logger,
    gradient_clip_val=1.0,
    accumulate_grad_batches=GRADIENT_ACCUMULATION,  # Memory-efficient training
    log_every_n_steps=10,
    val_check_interval=1.0,  # Validate every epoch
    enable_progress_bar=True,
    enable_model_summary=True,
    # Memory optimizations
    enable_checkpointing=True,
    deterministic=False  # Faster training
)

print("✅ Trainer configured")
print(f"   Devices: {trainer.num_devices} GPU")
print(f"   Precision: {trainer.precision}")
print(f"   Max epochs: {MAX_EPOCHS}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Gradient accumulation: {GRADIENT_ACCUMULATION}")
print(f"   Effective batch size: {BATCH_SIZE * GRADIENT_ACCUMULATION}")
print(f"   Learning rate: {LEARNING_RATE}")
print(f"   Memory cleanup: Every {EMPTY_CACHE_EVERY_N_STEPS} steps")

In [ ]:
# Start training
print("\n🚀 Starting fine-tuning...\n")
print("=" * 60)

# Clear memory before training
clear_memory()
print_memory_stats()

trainer.fit(asr_model)

# Clear memory after training
clear_memory()
print_memory_stats()

print("\n" + "=" * 60)
print("✅ Training complete!")

### 🔄 Resume Training (Optional)

If training was interrupted, you can resume from the last checkpoint:

```python
# Uncomment and run this instead of trainer.fit(asr_model)
# checkpoint_path = "/kaggle/working/checkpoints/last.ckpt"
# trainer.fit(asr_model, ckpt_path=checkpoint_path)
```

---

## 💾 Model Export

Save the fine-tuned model in NeMo format.

In [ ]:
# Save model to .nemo format
print("💾 Saving model...")

# Clear memory before saving
clear_memory()

asr_model.save_to(FINAL_MODEL_PATH)

print(f"✅ Model saved to: {FINAL_MODEL_PATH}")
print(f"   File size: {os.path.getsize(FINAL_MODEL_PATH) / (1024**3):.2f} GB")

# Delete trainer to free memory
del trainer
clear_memory()
print_memory_stats()

---

## 🎤 Test Data Inference & Submission

Process test audio files and generate submission CSV.

In [ ]:
# Load saved model for inference
print("📥 Loading saved model for inference...")

# Clear memory first
clear_memory()
print_memory_stats()

loaded_model = nemo_asr.models.ASRModel.restore_from(FINAL_MODEL_PATH)
loaded_model.eval()
loaded_model = loaded_model.cuda()  # Move to GPU

print("✅ Model loaded successfully")
print_memory_stats()

In [ ]:
# Quick validation test (optional)
# Test model on a validation sample before processing entire test set
if val_data and len(val_data) > 0:
    print("🧪 Quick model validation test...\n")
    
    sample = val_data[0]
    
    try:
        # Transcribe the full audio file
        prediction = loaded_model.transcribe([sample['audio_filepath']])[0]
        
        print(f"Ground Truth: {sample['text'][:200]}...")
        print(f"Prediction:   {prediction[:200]}...")
        print("\n✅ Model working correctly!")
        
    except Exception as e:
        print(f"❌ Model test failed: {str(e)}")
        print("   Check if model loaded correctly and paths are valid")
    
    clear_memory()
else:
    print("⚠️  No validation data available for testing")

In [ ]:
# Get all test audio files
print(f"🔍 Scanning test directory: {TEST_AUDIO_DIR}")

test_audio_files = sorted(Path(TEST_AUDIO_DIR).glob("*.wav"))
print(f"Found {len(test_audio_files)} test audio files")

if len(test_audio_files) == 0:
    print("⚠️  WARNING: No test audio files found! Check TEST_AUDIO_DIR path.")
else:
    # Check file sizes and durations
    total_duration = 0
    print("\n📊 Test dataset info:")
    sample_files = test_audio_files[:3]  # Show first 3
    for f in sample_files:
        duration = get_audio_duration(str(f))
        size_mb = f.stat().st_size / (1024 ** 2)
        total_duration += duration
        print(f"  {f.name}: {duration/60:.2f} min, {size_mb:.2f} MB")
    
    if len(test_audio_files) > 3:
        print(f"  ... and {len(test_audio_files) - 3} more files")
    
    # Estimate total duration
    for f in test_audio_files[3:]:
        total_duration += get_audio_duration(str(f))
    
    print(f"\n  Total test duration: {total_duration/3600:.2f} hours")
    print(f"  Average file duration: {total_duration/len(test_audio_files)/60:.2f} minutes")

In [ ]:
# Estimate inference time
if len(test_audio_files) > 0:
    # Rough estimate: ~2-3x real-time for P100 with long audio
    avg_duration_per_file = total_duration / len(test_audio_files)
    processing_time_per_file = avg_duration_per_file * 2.5  # Conservative estimate for long audio
    total_estimated_time = (processing_time_per_file * len(test_audio_files)) / 60  # in minutes
    
    print(f"\n⏱️  Estimated inference time:")
    print(f"   ~{total_estimated_time:.1f} minutes for {len(test_audio_files)} files")
    print(f"   (~{total_estimated_time/60:.1f} hours)")
    print(f"\n💡 Tip: Full audio files take longer to process. Monitor memory usage.\n")

In [ ]:
def transcribe_audio(
    model,
    audio_path: str,
    max_retries: int = 3
) -> str:
    """
    Transcribe a full audio file.
    
    Args:
        model: NeMo ASR model
        audio_path: Path to audio file
        max_retries: Number of retries on error
        
    Returns:
        Complete transcript
    """
    for attempt in range(max_retries):
        try:
            # Transcribe directly (NeMo handles long audio)
            transcript = model.transcribe([str(audio_path)])[0]
            return transcript
        except RuntimeError as e:
            if "out of memory" in str(e).lower():
                print(f"  ⚠️  OOM error on attempt {attempt + 1}/{max_retries}")
                clear_memory()
                if attempt < max_retries - 1:
                    import time
                    time.sleep(2)
                else:
                    raise
            else:
                raise
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"  ⚠️  Error on attempt {attempt + 1}/{max_retries}: {str(e)}")
                clear_memory()
                import time
                time.sleep(1)
            else:
                raise
    
    return ""


print("✅ Transcription function defined")

In [ ]:
# Process all test files
print("\n🎤 Starting test inference...\n")
print("Processing full audio files (no chunking)\n")

results = []

for audio_path in tqdm(test_audio_files, desc="Transcribing test files"):
    filename = audio_path.name
    
    try:
        # Transcribe full audio file
        transcript = transcribe_audio(
            loaded_model,
            str(audio_path),
            max_retries=3
        )
        
        # Store result
        results.append({
            "filename": filename,
            "transcript": transcript
        })
        
    except Exception as e:
        print(f"\n⚠️  Error processing {filename}: {str(e)}")
        # Add empty transcript for failed files
        results.append({
            "filename": filename,
            "transcript": ""
        })
    
    # Clear memory after each file
    clear_memory()

print(f"\n✅ Inference complete! Processed {len(results)} files")
print_memory_stats()

### 📊 Generate Submission File

In [ ]:
# Create submission DataFrame
print("📝 Creating submission file...")

submission_df = pd.DataFrame(results)
submission_df = submission_df[["filename", "transcript"]]

# Remove .wav extension from filenames
submission_df["filename"] = submission_df["filename"].str.replace(r"\.wav$", "", regex=True)

# Fill any empty transcriptions
submission_df["transcript"] = submission_df["transcript"].fillna("")

# Save submission
submission_csv_path = SUBMISSION_PATH + "submission.csv"
submission_df.to_csv(submission_csv_path, index=False, encoding="utf-8")

print(f"✅ Submission saved to: {submission_csv_path}")
print(f"   Total rows: {len(submission_df)}")

# Display preview
print(f"\n📄 Submission preview:")
print(submission_df.head(10))

# Statistics
print(f"\n📊 Submission statistics:")
print(f"   Files with transcripts: {(submission_df['transcript'] != '').sum()}")
print(f"   Empty transcripts: {(submission_df['transcript'] == '').sum()}")
print(f"   Average transcript length: {submission_df['transcript'].str.len().mean():.1f} characters")

In [ ]:
# Optional: Display some example transcriptions
print("\n🎯 Sample transcriptions:\n")

for i in range(min(5, len(submission_df))):
    row = submission_df.iloc[i]
    print(f"File: {row['filename']}")
    print(f"Transcript: {row['transcript'][:200]}{'...' if len(row['transcript']) > 200 else ''}")
    print("-" * 80)

# Clear memory at end
clear_memory()
print("\n✅ All done!")
print_memory_stats()

---

## 📊 Summary

### Completed Steps:

1. ✅ **Setup & Installation**: Installed NeMo toolkit and dependencies
2. ✅ **Transcript Loading**: Loaded full transcripts from individual .txt files (one per audio file)
3. ✅ **Audio Preprocessing**: Process full 40-minute audio files (NO chunking)
4. ✅ **Manifest Generation**: Created NeMo JSON manifests for full audio files
5. ✅ **Model Fine-tuning**: Fine-tuned pretrained Bangla Conformer on P100 with full audio files
6. ✅ **Model Export**: Saved model to `.nemo` format
7. ✅ **Test Inference**: Processed all test audio files (full files, no chunking)
8. ✅ **Submission Generation**: Created CSV file ready for submission

### 🎯 Data Processing Features:

- 📂 **Individual Transcript Files**: Each audio file (e.g., `audio1.wav`) has a corresponding transcript file (`audio1.txt`)
- 🎵 **Full Audio Processing**: Train on complete 40-minute audio files with full transcripts
- 📊 **Configurable Dataset Size**: Use `USE_FIRST_50_PERCENT` to train on subset of data (time optimization)
- ✅ **Perfect Alignment**: No transcript splitting - full audio paired with full transcript
- 🎯 **Batch Size = 1**: One full audio file per batch for memory management

### 💾 Memory Optimization Features:

- ⚡ **Gradient Accumulation**: Simulates batch_size=8 while using batch_size=1
- ⚡ **Mixed Precision (FP16)**: Reduces memory usage by ~50%
- ⚡ **Gradient Checkpointing**: Trades computation for memory
- ⚡ **Aggressive Cache Clearing**: Clears memory after each inference
- ⚡ **Lower Learning Rate**: More stable training with long sequences
- ⚡ **Pin Memory Disabled**: Reduces CPU memory overhead
- 📉 **50% Data Option**: Train on first half of dataset for faster iterations

### 🎯 Key Benefits:

- ✅ **Perfect Audio-Transcript Alignment**: No chunking misalignment issues
- ✅ **Simpler Pipeline**: No transcript splitting or audio segmentation needed
- ✅ **Better Context**: Model trains on full context of long audio
- ✅ **Production ready**: Complete inference pipeline with error handling and retries
- ⚠️ **Memory Intensive**: Requires careful memory management for 40-minute audio files

### 📁 Output Files:

- **Train Manifest**: `/kaggle/working/processed_data/manifests/train_manifest.json`
- **Val Manifest**: `/kaggle/working/processed_data/manifests/val_manifest.json`
- **Checkpoints**: `/kaggle/working/checkpoints/`
- **Final Model**: `/kaggle/working/nemo_bangla_asr_finetuned.nemo`
- **Submission**: `/kaggle/working/submission.csv`

---

### 🚀 Next Steps:

1. ✅ Download `submission.csv` for competition submission
2. 💡 Monitor TensorBoard logs for training insights
3. 🔧 Fine-tune hyperparameters if needed (learning rate, batch size, etc.)
4. 📊 Evaluate WER on validation set
5. 🎯 Further optimize for better accuracy
6. 🔄 If needed, train on full dataset by setting `USE_FIRST_50_PERCENT = False`

---

### 📝 Notes:

- 🎵 **Audio Format**: Processing FULL 40-minute audio files (NO chunking)
- 📄 **Transcript Format**: Each audio file needs ONE corresponding .txt file with full transcript
- 📊 **Dataset Size**: Currently using first 50% of training data (controlled by `USE_FIRST_50_PERCENT`)
- ✅ **Perfect Alignment**: Full audio paired with full transcript - no splitting required
- 🎯 **Batch Size**: MUST be 1 for memory management with long audio files
- ⚡ **Gradient Accumulation**: Set to 8 to simulate batch_size=8
- 💾 **Memory**: Long audio files (~40 min) are memory-intensive
- ⚠️ **Duration Limit**: Files longer than 1 hour are skipped (configurable via `MAX_AUDIO_DURATION`)
- 🔧 **Error Handling**: Empty transcripts and failed files are handled gracefully

---